In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from util import prepare_df

In [ ]:
df = prepare_df(
    "../outputs/20240214_210624_oracles_header.csv",
    "../outputs/20240214_210624_oracles_explo.csv",
)

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
task_palette = sns.color_palette("deep", int(df["n_task"].unique().shape[0]))
sns.set_theme(style="white", palette="deep", rc={"figure.figsize": (11.7, 8.27)})

In [ ]:
df["Number of tasks"] = df["nbt"]
df["Schedulable"] = df["schedulable"]

In [ ]:
df["oracle"] = df["safe"].combine_first(df["unsafe"])
df["oracle"] = df["oracle"].fillna("none")

In [ ]:
df.groupby("tid")["schedulable"].min().value_counts()

In [ ]:
print(df["duration_s"].sum() / 60)
print(df["duration_s"].sum() / 3600)

In [ ]:
df_root = df[
    ["tid", "n_task", "schedulable", "Number of tasks", "Schedulable"]
].drop_duplicates()
df_root = df_root.set_index("tid")

In [ ]:
df_ss = df.pivot(index="tid", columns="oracle", values=["duration_s", "n_visited"])
df_ss.columns = list(map(lambda x: "_".join(x), df_ss.columns))
df_ss = df_root.join(df_ss)

In [ ]:
oracle_sort_list = (
    df.groupby("oracle")["n_visited"].median().sort_values().index.to_list()[::-1]
)

In [ ]:
g = sns.boxplot(
    data=df,
    y="n_visited",
    x="n_task",
    hue="oracle",
    hue_order=df.groupby("oracle")["n_visited"]
    .median()
    .sort_values()
    .index.to_list()[::-1],
)
g.set_yscale("log")
plt.show()

In [ ]:
g = sns.boxplot(
    data=df,
    y="duration_s",
    x="n_task",
    hue="oracle",
    hue_order=df.groupby("oracle")["duration_s"]
    .median()
    .sort_values()
    .index.to_list()[::-1],
)
g.set_yscale("log")
plt.show()

In [ ]:
df_comp = df.loc[df["oracle"] != "none"].merge(
    df.loc[df["oracle"] == "none", ["tid", "duration_s", "n_visited"]],
    on="tid",
    suffixes=("", "_none"),
)
df_comp["n_visited_ratio"] = df_comp["n_visited"] / df_comp["n_visited_none"]
df_comp["duration_s_ratio"] = df_comp["duration_s"] / df_comp["duration_s_none"]
df_comp["n_visited_change"] = df_comp["n_visited_ratio"] - 1
df_comp["duration_s_change"] = df_comp["duration_s_ratio"] - 1

In [ ]:
df_comp_melt = df_comp.melt(
    id_vars=["tid", "oracle", "schedulable"],
    value_vars=["n_visited_change", "duration_s_change"],
)


df_comp_melt = df_comp_melt.rename(
    columns={
        "variable": "Dimension",
        "value": "Relative change",
        "schedulable": "Schedulable",
        "oracle": "Oracle",
    }
)

df_comp_melt["Relative change"] = df_comp_melt["Relative change"] * 100

metric_map = {
    "duration_s_change": "Execution time (s)",
    "n_visited_change": "Number of visited states before halt",
}

df_comp_melt["Dimension"] = df_comp_melt["Dimension"].map(lambda x: metric_map[x])

oracle_map = {
    "all_idle_hi": "All idle HI",
    "all_interference": "All interference",
    "interference": "Interference",
    "none": "None",
    "laxity": "Laxity",
    "hi_interference": "HI-Interference",
    "worst_interference": "HI-Interference",
    "worst_laxity": "Worst Laxity",
}

df_comp_melt["Oracle"] = df_comp_melt["Oracle"].map(lambda x: oracle_map[x])

In [ ]:
fg = sns.catplot(
    df_comp_melt,
    kind="box",
    x="Relative change",
    y="Oracle",
    col="Dimension",
    hue="Schedulable",
    height=5.5129,
    aspect=1,
    sharex=False,
    col_order=["Number of visited states before halt", "Execution time (s)"],
    showfliers=False,
)

In [ ]:
import matplotlib.ticker as mtick

In [ ]:
order = [
    "All idle HI",
    "Laxity",
    "Interference",
    "Worst Laxity",
    "All interference",
    "HI-Interference",
]

fg = sns.catplot(
    df_comp_melt,
    kind="bar",
    y="Relative change",
    x="Oracle",
    col="Dimension",
    hue="Schedulable",
    height=5.5129,
    aspect=1,
    sharex=False,
    col_order=["Number of visited states before halt", "Execution time (s)"],
    errorbar=None,
    estimator="median",
    order=order,
)


fg.set_titles("{col_name}")

# draw a dashed vertical line at 0 on the first plot of the face grid
fg.axes[0, 1].axhline(0, color="k", linestyle="--")
fg.axes[0, 0].axhline(0, color="k", linestyle="--")

fmt_f = lambda x: ("+" if x > 0 else "") + f"{x:.1f}%"

fg.axes[0, 0].bar_label(fg.axes[0, 0].containers[0], fontsize=8, fmt=fmt_f)
fg.axes[0, 0].bar_label(fg.axes[0, 0].containers[1], fontsize=8, fmt=fmt_f)
fg.axes[0, 1].bar_label(fg.axes[0, 1].containers[0], fontsize=8, fmt=fmt_f)
fg.axes[0, 1].bar_label(fg.axes[0, 1].containers[1], fontsize=8, fmt=fmt_f)

fg.axes[0, 0].yaxis.set_major_formatter(mtick.PercentFormatter(xmax=100))


sns.despine(fg.figure, bottom=True)

fg.set_xticklabels(rotation=30)

fg.savefig(
    f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_oracle.pdf", bbox_inches="tight"
)

In [ ]:
fg = sns.displot(
    df_comp_melt,
    # kind="kde",
    x="Relative change",
    row="Oracle",
    col="Dimension",
    hue="Schedulable",
    height=2,
    aspect=3,
    col_order=["Number of visited states before halt", "Execution time (s)"],
    facet_kws=dict(sharey=False, xlim=(-105, 25)),
)